Another attempt at feature extraction this time using Fast Fourier and Wavelet Transform, as opposed to the fractal decompositions used earlier

Strong Reference:
Python Emotion using EEG:
    (https://github.com/shivam-199/Python-Emotion-using-EEG-Signal)

In [48]:
import os
import sys

import pandas as pd
import numpy as np

import pywt
import scipy.io as spio
from scipy.stats import entropy
from collections import Counter

from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import timeit

In [49]:
WAVELET = "db6"
MAX_LEVEL = 5

In [51]:
channel_order = pd.read_csv(r"C:\Users\tanya\OneDrive\Documents\GitHub\LVO-EEG\data\feature_processing\001_EEG_baseline_stroke_study_updated.csv")
channel_order.head()

,TP9,AF7,AF8,TP10
0,23.926,11.719,121.094,22.949
1,22.461,39.062,-9.277,24.414
2,23.926,37.109,-19.531,23.926
3,38.574,28.809,47.363,41.992
4,52.734,66.895,51.758,57.617


In [110]:
def calc_wavelet_energy(data_set):
  """
    Input : 1 * N vector
    Output: Float with the wavelet energy of the input vector,
    rounded to 3 decimal places.
  """
  clean = np.square(data_set)
  clean = np.where(clean == 0, 1, clean)
  wavelet_energy = np.nansum(np.log2(clean))
  return round(wavelet_energy, 3)

In [111]:
def calc_shannon_entropy(data_set):
        """
            Input : 1 * N vector
            Output: Float with the wavelet entropy of the input vector,
                    rounded to 3 decimal places.
        """
        probability = np.square(data_set)
        probability = np.where(probability == 0, 1, probability)
        shannon_entropy = -np.nansum(probability * np.log2(probability))
        return round(shannon_entropy, 3)

In [112]:
files = os.listdir(r"C:\Users\tanya\OneDrive\Documents\GitHub\LVO-EEG\data\feature_processing")
files


['001_EEG_baseline_stroke_study_updated.csv',
 '002_EEG_baseline_stroke_study_updated.csv',
 '003_EEG_baseline_stroke_study_updated.csv',
 '004_EEG_baseline_stroke_study_updated.csv',
 '005_EEG_baseline_stroke_study_updated.csv',
 '006_EEG_baseline_stroke_study_updated.csv',
 '007_EEG_baseline_stroke_study_updated.csv',
 '008_EEG_baseline_stroke_study_updated.csv',
 '009_EEG_baseline_stroke_study_updated.csv',
 '010_EEG_baseline_stroke_study_updated.csv',
 '011_EEG_baseline_stroke_study_updated.csv',
 '012_EEG_baseline_stroke_study_updated.csv',
 '013_EEG_baseline_stroke_study_updated.csv',
 '014_EEG_baseline_stroke_study_updated.csv',
 '015_EEG_baseline_stroke_study_updated.csv',
 '016_EEG_baseline_stroke_study_updated.csv',
 '017_EEG_baseline_stroke_study_updated.csv',
 '018_EEG_baseline_stroke_study_updated.csv',
 '019_EEG_baseline_stroke_study_updated.csv',
 '020_EEG_baseline_stroke_study_updated.csv',
 '021_EEG_baseline_stroke_study_updated.csv',
 '022_EEG_baseline_stroke_study_up

In [113]:
WAVELET = "db6"
MAX_LEVEL = 5 # max level of wavelet decomposition 5 bands

In [114]:
participant_trial = []
features_table = pd.DataFrame(columns=range(40))
for file in files:#for each participant
    df = pd.read_csv(r"C:\Users\tanya\OneDrive\Documents\GitHub\LVO-EEG\data\feature_processing\\"+file)
    wavEnergy, shannonEntropy = [], []
    for column in df: #for each channel
        oneColumn = df[column].to_numpy()
        dwt_bands = []
        int_ent, int_eng = [], []
        for band in range(MAX_LEVEL):
            (cA, cD) = pywt.dwt(oneColumn, WAVELET)
            dwt_bands.append(cD)   # Append the approximations
        for band in range(len(dwt_bands)):
            int_ent.append(calc_shannon_entropy(dwt_bands[len(dwt_bands) - band - 1]))
            int_eng.append(calc_wavelet_energy(dwt_bands[len(dwt_bands) - band - 1]))
        wavEnergy.append(int_eng)
        shannonEntropy.append(int_ent)
    unroll_entropy = []
    unroll_energy = []
    '''
        Transforming 2D array into 1D vector of features and then 
        combining the two 1D arrays.
    '''
    for i in range(len(shannonEntropy)):
      for j in range(len(shannonEntropy[0])):
        unroll_entropy.append(shannonEntropy[i][j])
        unroll_energy.append(wavEnergy[i][j])

    features = unroll_entropy + unroll_energy
    participant_trial.append(features)
    features_table.loc[len(features_table.index)] = features
    # print(data_file)
    # print(features)
    print(file)


001_EEG_baseline_stroke_study_updated.csv
002_EEG_baseline_stroke_study_updated.csv
003_EEG_baseline_stroke_study_updated.csv
004_EEG_baseline_stroke_study_updated.csv
005_EEG_baseline_stroke_study_updated.csv
006_EEG_baseline_stroke_study_updated.csv
007_EEG_baseline_stroke_study_updated.csv
008_EEG_baseline_stroke_study_updated.csv
009_EEG_baseline_stroke_study_updated.csv
010_EEG_baseline_stroke_study_updated.csv
011_EEG_baseline_stroke_study_updated.csv
012_EEG_baseline_stroke_study_updated.csv
013_EEG_baseline_stroke_study_updated.csv
014_EEG_baseline_stroke_study_updated.csv
015_EEG_baseline_stroke_study_updated.csv
016_EEG_baseline_stroke_study_updated.csv
017_EEG_baseline_stroke_study_updated.csv
018_EEG_baseline_stroke_study_updated.csv
019_EEG_baseline_stroke_study_updated.csv
020_EEG_baseline_stroke_study_updated.csv
021_EEG_baseline_stroke_study_updated.csv
022_EEG_baseline_stroke_study_updated.csv
023_EEG_baseline_stroke_study_updated.csv
024_EEG_baseline_stroke_study_upda

In [115]:
file_dir = r"C:\Users\tanya\OneDrive\Documents\GitHub\LVO-EEG\data\feature_processed"

In [116]:
features_table.to_csv(file_dir + "features" + WAVELET + ".csv", index=False)

In [117]:
features_table.to_csv(r"C:\Users\tanya\OneDrive\Documents\GitHub\LVO-EEG\data\feature_processed"+r"\features.csv", index=False)

In [118]:
data = pd.read_csv(r"C:\Users\tanya\OneDrive\Documents\GitHub\LVO-EEG\data\feature_processed"+r"\features.csv")